In [1]:
pip install pydantic

In [2]:
pip install "pydantic-ai-slim[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.1/291.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.9 MB/s eta 0:00:00


In [3]:
pip install nest-asyncio

In [4]:
!pip install devtools
!pip install logfire

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.8/213.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00


In [5]:
from pydantic import BaseModel, PositiveFloat

class ProductQuery(BaseModel):
    product_name: str
    max_price: PositiveFloat

# ข้อมูลถูกต้องตามเงื่อนไขที่กำหนด
query = ProductQuery(product_name="กาแฟสด", max_price=1000.0)
print(query)


product_name='กาแฟสด' max_price=1000.0


In [6]:
query = ProductQuery(product_name="กาแฟสด", max_price=0.0)


ValidationError: 1 validation error for ProductQuery
max_price
  Input should be greater than 0 [type=greater_than, input_value=0.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than

In [7]:
query = ProductQuery(product_name="กาแฟสด", max_price=-500.0)

ValidationError: 1 validation error for ProductQuery
max_price
  Input should be greater than 0 [type=greater_than, input_value=-500.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than

In [8]:
import os
from google.colab import userdata
groq_key = userdata.get('GROQ_CHULA')

os.environ["GROQ_API_KEY"] = groq_key



In [9]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
from pydantic_ai import Agent


agent = Agent(
    'groq:openai/gpt-oss-20b',
    system_prompt='Be concise, reply with one sentence.',
)


result = agent.run_sync('Where is the capital of England')
print(result.output)


The capital of England is London.


In [13]:
from pydantic_ai import Agent


agent = Agent(
    'groq:openai/gpt-oss-20b',
    system_prompt='Be concise, reply in Thai with one sentence.',
)


result = agent.run_sync('Where is the capital of England')
print(result.output)


ลอนดอนเป็นเมืองหลวงของอังกฤษและตั้งอยู่ที่ส่วนตะวันออกเฉียงใต้ของประเทศบนแม่น้ำเทมส์.


In [14]:
import os
from google.colab import userdata

os.environ["WEATHER_API_KEY"] = userdata.get('WEATHER_API_KEY').strip()
os.environ["GEO_API_KEY"] = userdata.get('GEO_API_KEY').strip()


In [21]:


from __future__ import annotations as _annotations

import asyncio
import os
from dataclasses import dataclass
from typing import Any

import logfire
from devtools import debug
from httpx import AsyncClient

from pydantic_ai import Agent, ModelRetry, RunContext


logfire.configure(send_to_logfire='if-token-present')


@dataclass
class Deps:
    client: AsyncClient
    weather_api_key: str | None
    geo_api_key: str | None


weather_agent = Agent(
    'groq:openai/gpt-oss-20b',
    system_prompt='Be concise, reply in Thai with one sentence.',
)


@weather_agent.tool
async def get_lat_lng(
    ctx: RunContext[Deps], location_description: str
) -> dict[str, float]:
    """Get the latitude and longitude of a location.

    Args:
        ctx: The context.
        location_description: A description of a location.
    """
    if ctx.deps.geo_api_key is None:
        return {'lat': 51.1, 'lng': -0.1}

    params = {
        'q': location_description,
        'api_key': ctx.deps.geo_api_key,
    }
    with logfire.span('calling geocode API', params=params) as span:
        r = await ctx.deps.client.get('https://geocode.maps.co/search', params=params)
        r.raise_for_status()
        data = r.json()
        span.set_attribute('response', data)

    if data:
        return {'lat': data[0]['lat'], 'lng': data[0]['lon']}
    else:
        raise ModelRetry('Could not find the location')


@weather_agent.tool
async def get_weather(ctx: RunContext[Deps], lat: float, lng: float) -> dict[str, Any]:
    """Get the weather at a location.

    Args:
        ctx: The context.
        lat: Latitude of the location.
        lng: Longitude of the location.
    """
    if ctx.deps.weather_api_key is None:
        # if no API key is provided, return a dummy response
        return {'temperature': '21 °C', 'description': 'Sunny'}

    params = {
        'apikey': ctx.deps.weather_api_key,
        'location': f'{lat},{lng}',
        'units': 'metric',
    }
    with logfire.span('calling weather API', params=params) as span:
        r = await ctx.deps.client.get(
            'https://api.tomorrow.io/v4/weather/realtime', params=params
        )
        r.raise_for_status()
        data = r.json()
        span.set_attribute('response', data)

    values = data['data']['values']
    # https://docs.tomorrow.io/reference/data-layers-weather-codes
    code_lookup = {
        1000: 'Clear, Sunny',
        1100: 'Mostly Clear',
        1101: 'Partly Cloudy',
        1102: 'Mostly Cloudy',
        1001: 'Cloudy',
        2000: 'Fog',
        2100: 'Light Fog',
        4000: 'Drizzle',
        4001: 'Rain',
        4200: 'Light Rain',
        4201: 'Heavy Rain',
        5000: 'Snow',
        5001: 'Flurries',
        5100: 'Light Snow',
        5101: 'Heavy Snow',
        6000: 'Freezing Drizzle',
        6001: 'Freezing Rain',
        6200: 'Light Freezing Rain',
        6201: 'Heavy Freezing Rain',
        7000: 'Ice Pellets',
        7101: 'Heavy Ice Pellets',
        7102: 'Light Ice Pellets',
        8000: 'Thunderstorm',
    }
    return {
        'temperature': f'{values["temperatureApparent"]:0.0f}°C',
        'description': code_lookup.get(values['weatherCode'], 'Unknown'),
    }


async def main():
    async with AsyncClient() as client:
        # create a free API key at https://www.tomorrow.io/weather-api/
        weather_api_key = os.getenv('WEATHER_API_KEY')
        # create a free API key at https://geocode.maps.co/
        geo_api_key = os.getenv('GEO_API_KEY')
        deps = Deps(
            client=client, weather_api_key=weather_api_key, geo_api_key=geo_api_key
        )
        result = await weather_agent.run(
            'What is the weather like in Bangkok and in New Jersey?', deps=deps
        )
        debug(result)
        print('Response:', result.output)


if __name__ == '__main__':
    asyncio.run(main())

06:40:19.683 calling geocode API
06:40:20.388 calling weather API
06:40:21.366 calling geocode API
06:40:21.998 calling weather API
/tmp/ipython-input-650333967.py:131 main
    result: AgentRunResult(
        output='กรุงเทพฯ 33\u202f°C หมอกเบา ในนิวเจอร์ซีย์ 14\u202f°C แดดจ้า',
        _output_tool_name=None,
        _state=GraphAgentState(
            message_history=[
                ModelRequest(
                    parts=[
                        SystemPromptPart(
                            content='Be concise, reply in Thai with one sentence.',
                            timestamp=datetime.datetime(2025, 8, 22, 6, 40, 19, 181900, tzinfo=datetime.timezone.utc),
                            dynamic_ref=None,
                            part_kind='system-prompt',
                        ),
                        UserPromptPart(
                            content='What is the weather like in Bangkok and in New Jersey?',
                            timestamp=datetime.datetime(2025,